## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import os, sys
currentdir = os.path.dirname(os.path.realpath("Vacation_Search"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.15,76,100,13.56,overcast clouds
1,1,Marsabit,KE,2.3284,37.9899,69.31,45,14,5.39,few clouds
2,2,Vaini,TO,-21.2000,-175.2000,80.60,89,75,4.61,broken clouds
3,3,Meulaboh,ID,4.1363,96.1285,74.77,89,100,4.90,overcast clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,55.40,47,75,10.36,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.15,76,100,13.56,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,80.60,89,75,4.61,broken clouds
9,9,Carnarvon,AU,-24.8667,113.6333,80.60,74,52,13.80,broken clouds
11,11,Saint-Philippe,RE,-21.3585,55.7679,77.00,83,0,10.36,light rain
19,19,Noumea,NC,-22.2763,166.4572,81.00,100,90,2.30,overcast clouds
20,20,Lorengau,PG,-2.0226,147.2712,80.94,67,100,3.24,overcast clouds
24,24,Bengkulu,ID,-3.8004,102.2655,75.65,85,100,4.14,overcast clouds
32,32,Campina Verde,BR,-19.5356,-49.4864,91.09,39,100,7.25,overcast clouds
33,33,Cabo San Lucas,MX,22.8909,-109.9124,75.00,46,1,9.22,clear sky
34,34,Esperance,AU,-33.8667,121.9000,77.00,38,40,19.57,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.15,overcast clouds,-23.1203,-134.9692,
2,Vaini,TO,80.60,broken clouds,-21.2000,-175.2000,
9,Carnarvon,AU,80.60,broken clouds,-24.8667,113.6333,
11,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
19,Noumea,NC,81.00,overcast clouds,-22.2763,166.4572,
20,Lorengau,PG,80.94,overcast clouds,-2.0226,147.2712,
24,Bengkulu,ID,75.65,overcast clouds,-3.8004,102.2655,
32,Campina Verde,BR,91.09,overcast clouds,-19.5356,-49.4864,
33,Cabo San Lucas,MX,75.00,clear sky,22.8909,-109.9124,
34,Esperance,AU,77.00,scattered clouds,-33.8667,121.9000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.15,overcast clouds,-23.1203,-134.9692,Pension Maro'i
2,Vaini,TO,80.60,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Carnarvon,AU,80.60,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
11,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
19,Noumea,NC,81.00,overcast clouds,-22.2763,166.4572,Hôtel Le Lagon
20,Lorengau,PG,80.94,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
24,Bengkulu,ID,75.65,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
32,Campina Verde,BR,91.09,overcast clouds,-19.5356,-49.4864,Hotel Lino
33,Cabo San Lucas,MX,75.00,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
34,Esperance,AU,77.00,scattered clouds,-33.8667,121.9000,Hospitality Esperance


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.15,overcast clouds,-23.1203,-134.9692,Pension Maro'i
2,Vaini,TO,80.60,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Carnarvon,AU,80.60,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
11,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
19,Noumea,NC,81.00,overcast clouds,-22.2763,166.4572,Hôtel Le Lagon
20,Lorengau,PG,80.94,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
24,Bengkulu,ID,75.65,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
32,Campina Verde,BR,91.09,overcast clouds,-19.5356,-49.4864,Hotel Lino
33,Cabo San Lucas,MX,75.00,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
34,Esperance,AU,77.00,scattered clouds,-33.8667,121.9000,Hospitality Esperance


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))